In [28]:
import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from io import BytesIO
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Dataset paths
folder_train_path = "extracted_data/Combined_dataset/train/"
folder_test_path = "extracted_data/Combined_dataset/test/"

# Output path for combined dataset
output_dir = "N_Combined_MRI_Dataset"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "test"), exist_ok=True)

# Settings
img_size = 128  # set 128 for now, might up to 256

# Class mapping - Aligning with the shared code
classes = {
    'Mild_Impairment': 0,
    'Moderate_Impairment': 1,
    'No_Impairment': 2,
    'Very_Mild_Impairment': 3
}

def load_and_normalize_dataset(folder_train_path, folder_test_path):
    normalized_data = []
    
    # Process train and test folders
    for split, path in [('train', folder_train_path), ('test', folder_test_path)]:
        if not os.path.exists(path):
            print(f"Warning: {path} does not exist, skipping...")
            continue
        
        # Process each class folder
        for class_name, label in classes.items():
            class_dir = os.path.join(path, class_name)
            
            if not os.path.exists(class_dir):
                print(f"Warning: Class directory {class_dir} does not exist, skipping...")
                continue
                
            # Process each image in the class folder
            print(f"Processing {split}/{class_name}...")
            for img_file in os.listdir(class_dir):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_dir, img_file)
                    
                    try:
                        # Load image
                        img = Image.open(img_path).convert('L')
                        img_array = np.array(img) / 255.0  # Normalize to [0,1]
                        
                        # Resize to standard size
                        img_resized = cv2.resize(img_array, (img_size, img_size))
                        
                        # Process and store
                        item = {
                            'original_path': img_path,
                            'label': label,
                            'class_name': class_name,
                            'split': split,
                            'image': img_resized,
                            'dataset': 'folder_dataset' if 'folder' in img_file else 'parquet_dataset'
                        }
                        
                        # Apply normalizations
                        normalized_item = normalize_mri_for_ventricles(item)
                        normalized_data.append(normalized_item)
                        
                    except Exception as e:
                        print(f"Error processing {img_path}: {e}")
    
    # Split back into train and test
    train_data = [item for item in normalized_data if item['split'] == 'train']
    test_data = [item for item in normalized_data if item['split'] == 'test']
    
    return train_data, test_data

def normalize_mri_for_ventricles(item):
    image = item['image']
    
    # 1. Simple normalization - just use the original normalized image
    item['image_normalized'] = image
    
    # 2. Ventricle enhancement
    
    # Create version optimized for dark ventricle regions
    img_uint8 = (item['image_normalized'] * 255).astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(img_uint8)
    item['image_enhanced'] = enhanced / 255.0
    
    # Create inverted version to highlight ventricles
    inverted = 1 - item['image_normalized']
    # Apply adaptive thresholding to highlight ventricle regions
    item['image_ventricle_focus'] = inverted
    
    # 3. Ventricle segmentation using Otsu thresholding
    otsu_thresh, _ = cv2.threshold(
        img_uint8, 
        0, 
        255, 
        cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU
    )
    
    _, ventricle_mask = cv2.threshold(
        img_uint8, 
        int(otsu_thresh * 0.5),  # may add 50% of the Otsu threshold
        255, 
        cv2.THRESH_BINARY_INV
    )
    
    # Clean up mask
    kernel = np.ones((3, 3), np.uint8)
    ventricle_mask = cv2.morphologyEx(ventricle_mask, cv2.MORPH_OPEN, kernel)
    ventricle_mask = cv2.morphologyEx(ventricle_mask, cv2.MORPH_CLOSE, kernel)
    
    item['ventricle_mask'] = ventricle_mask / 255.0
    
    return item

def visualize_normalizations(data, num_samples=4):
    # Select samples from each class if possible
    samples = []
    class_names = set(item['class_name'] for item in data)
    
    for class_name in class_names:
        class_items = [item for item in data if item['class_name'] == class_name]
        if class_items:
            samples.append(class_items[0])
            if len(samples) >= num_samples:
                break
    
    # Use random samples if we don't have enough
    if len(samples) < num_samples:
        remaining = [i for i in data if not any(s['original_path'] == i['original_path'] for s in samples)]
        if remaining:
            additional = np.random.choice(
                remaining,
                size=min(num_samples - len(samples), len(remaining)),
                replace=False
            ).tolist()
            samples.extend(additional)
    
    # Create visualization
    fig, axes = plt.subplots(len(samples), 5, figsize=(20, 4 * len(samples)))
    
    # Handle case with just one sample
    if len(samples) == 1:
        axes = [axes]
    
    for i, item in enumerate(samples):
        # Original
        axes[i][0].imshow(item['image'], cmap='gray')
        axes[i][0].set_title(f"{item['class_name']}\nOriginal")
        axes[i][0].axis('off')
        
        # Normalized
        axes[i][1].imshow(item['image_normalized'], cmap='gray')
        axes[i][1].set_title('Normalized')
        axes[i][1].axis('off')
        
        # Enhanced
        axes[i][2].imshow(item['image_enhanced'], cmap='gray')
        axes[i][2].set_title('Enhanced (CLAHE)')
        axes[i][2].axis('off')
        
        # Ventricle Focus (Inverted)
        axes[i][3].imshow(item['image_ventricle_focus'], cmap='gray')
        axes[i][3].set_title('Ventricle Focus')
        axes[i][3].axis('off')
        
        # Ventricle Mask
        axes[i][4].imshow(item['ventricle_mask'], cmap='gray')
        axes[i][4].set_title('Ventricle Mask')
        axes[i][4].axis('off')
    
    plt.tight_layout()
    return fig

def save_normalized_dataset(output_dir, train_data, test_data):
    saved_count = 0
    
    for split, data in [('train', train_data), ('test', test_data)]:
        split_dir = os.path.join(output_dir, split)
        os.makedirs(split_dir, exist_ok=True)
        
        # Create class directories
        class_names = set(item['class_name'] for item in data)
        for class_name in class_names:
            os.makedirs(os.path.join(split_dir, class_name), exist_ok=True)
        
        # Save normalized images
        for item in tqdm(data, desc=f"Saving {split} images"):
            # Generate filename
            original_filename = os.path.basename(item['original_path'])
            base_name = os.path.splitext(original_filename)[0]
            
            # Define paths for different normalizations
            class_dir = os.path.join(split_dir, item['class_name'])
            
            # Save normalized image
            norm_img = (item['image_normalized'] * 255).astype(np.uint8)
            norm_path = os.path.join(class_dir, f"{base_name}_norm.png")
            Image.fromarray(norm_img).save(norm_path)
            
            # Save enhanced image
            enhanced_img = (item['image_enhanced'] * 255).astype(np.uint8)
            enhanced_path = os.path.join(class_dir, f"{base_name}_enhanced.png")
            Image.fromarray(enhanced_img).save(enhanced_path)
            
            # Save ventricle focused image
            ventricle_img = (item['image_ventricle_focus'] * 255).astype(np.uint8)
            ventricle_path = os.path.join(class_dir, f"{base_name}_ventricle.png")
            Image.fromarray(ventricle_img).save(ventricle_path)
            
            # Save ventricle mask
            mask_img = (item['ventricle_mask'] * 255).astype(np.uint8)
            mask_path = os.path.join(class_dir, f"{base_name}_mask.png")
            Image.fromarray(mask_img).save(mask_path)
            
            saved_count += 1
    
    print(f"Total of {saved_count} images processed and saved with normalizations")

# Main execution
if __name__ == "__main__":
    print("Loading and normalizing dataset...")
    train_data, test_data = load_and_normalize_dataset(folder_train_path, folder_test_path)
    
    print(f"Processed {len(train_data)} training images and {len(test_data)} test images")
    
    # Class distribution summary
    for split_name, split_data in [("Training", train_data), ("Test", test_data)]:
        print(f"\n{split_name} set class distribution:")
        class_counts = {}
        for item in split_data:
            class_name = item['class_name']
            if class_name in class_counts:
                class_counts[class_name] += 1
            else:
                class_counts[class_name] = 1
        
        for class_name, count in class_counts.items():
            print(f"  {class_name}: {count} images ({count/len(split_data)*100:.1f}%)")
    
    # Visualize normalizations
    print("\nGenerating visualization...")
    fig = visualize_normalizations(train_data)
    plt.savefig(os.path.join(output_dir, "normalization_visualization.png"))
    plt.close(fig)
    
    # Save normalized dataset
    print("\nSaving normalized dataset...")
    save_normalized_dataset(output_dir, train_data, test_data)
    
    print("\nFeature extraction complete!")

Loading and normalizing dataset...
Processing train/Mild_Impairment...
Processing train/Moderate_Impairment...
Processing train/No_Impairment...
Processing train/Very_Mild_Impairment...
Processing test/Mild_Impairment...
Processing test/Moderate_Impairment...
Processing test/No_Impairment...
Processing test/Very_Mild_Impairment...
Processed 10240 training images and 1279 test images

Training set class distribution:
  Mild_Impairment: 2560 images (25.0%)
  Moderate_Impairment: 2560 images (25.0%)
  No_Impairment: 2560 images (25.0%)
  Very_Mild_Impairment: 2560 images (25.0%)

Test set class distribution:
  Mild_Impairment: 179 images (14.0%)
  Moderate_Impairment: 12 images (0.9%)
  No_Impairment: 640 images (50.0%)
  Very_Mild_Impairment: 448 images (35.0%)

Generating visualization...

Saving normalized dataset...


Saving test images: 100%|██████████████████| 1279/1279 [00:01<00:00, 726.29it/s]

Total of 11519 images processed and saved with normalizations

Feature extraction complete!
